In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/module/FaceDetector')

In [17]:
%%writefile boundingbox.py


import numpy as np
import sys
import getopt
import cv2
import os


class bounding_box:
	def __init__(self, xmin, ymin, xmax, ymax, c=None, classes=None):
		self.xmin = xmin
		self.ymin = ymin
		self.xmax = xmax
		self.ymax = ymax

		self.c = c
		self.classes = classes

		self.label = -1
		self.score = -1

	def get_label(self):
		if self.label == -1:
			self.label = np.argmax(self.classes)

		return self.label

	def get_score(self):
		if self.score == -1:
			self.score = self.classes[self.get_label()]

		return self.score


def bbox_iou(box1, box2):
	intersect_w = _interval_overlap(
		[box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
	intersect_h = _interval_overlap(
		[box1.ymin, box1.ymax], [box2.ymin, box2.ymax])

	intersect = intersect_w * intersect_h

	w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
	w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin

	union = w1*h1 + w2*h2 - intersect

	return float(intersect) / union


def _interval_overlap(interval_a, interval_b):
	x1, x2 = interval_a
	x3, x4 = interval_b

	if x3 < x1:
		if x4 < x1:
			return 0
		else:
			return min(x2, x4) - x1
	else:
		if x2 < x3:
			 return 0
		else:
			return min(x2, x4) - x3


def _sigmoid(x):
	return 1. / (1. + np.exp(-x))


def _softmax(x, axis=-1, t=-100.):
	x = x - np.max(x)

	if np.min(x) < t:
		x = x/np.min(x)*t

	e_x = np.exp(x)

	return e_x / e_x.sum(axis, keepdims=True)


#crop
def crop(x, y, w, h, margin, img_width, img_height):
  if h> w: 
    w = h
  else:
    h = w

  xmin = int(x-w*margin)
  xmax = int(x+w*margin)
  ymin = int(y-h*margin)
  ymax = int(y+h*margin)
  
  if xmin < 0:
    xmin = 0
  if ymin < 0:
    ymin = 0
  if xmax > img_width:
    xmax = img_width
  if ymax > img_height:
    ymax = img_height
  return xmin, xmax, ymin, ymax


def interpret_output_yolov2(output, img_width, img_height):
	anchors = [0.57273, 0.677385, 1.87446, 2.06253,
            3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828]

	netout = output
	nb_class = 1
	obj_threshold = 0.4
	nms_threshold = 0.3

	grid_h, grid_w, nb_box = netout.shape[:3]

	size = 4 + nb_class + 1
	nb_box = 5

	netout = netout.reshape(grid_h, grid_w, nb_box, size)

	boxes = []

	# decode the output by the network
	netout[..., 4] = _sigmoid(netout[..., 4])
	netout[..., 5:] = netout[..., 4][..., np.newaxis] * _softmax(netout[..., 5:])
	netout[..., 5:] *= netout[..., 5:] > obj_threshold

	for row in range(grid_h):
		for col in range(grid_w):
			for b in range(nb_box):
				# from 4th element onwards are confidence and class classes
				classes = netout[row, col, b, 5:]

				if np.sum(classes) > 0:
					# first 4 elements are x, y, w, and h
					x, y, w, h = netout[row, col, b, :4]

					x = (col + _sigmoid(x)) / grid_w  # center position, unit: image width
					y = (row + _sigmoid(y)) / grid_h  # center position, unit: image height
					w = anchors[2 * b + 0] * np.exp(w) / grid_w  # unit: image width
					h = anchors[2 * b + 1] * np.exp(h) / grid_h  # unit: image height
					confidence = netout[row, col, b, 4]

					box = bounding_box(x-w/2, y-h/2, x+w/2, y+h/2, confidence, classes)

					boxes.append(box)

	# suppress non-maximal boxes
	for c in range(nb_class):
		sorted_indices = list(
			reversed(np.argsort([box.classes[c] for box in boxes])))

		for i in range(len(sorted_indices)):
			index_i = sorted_indices[i]

			if boxes[index_i].classes[c] == 0:
				continue
			else:
				for j in range(i+1, len(sorted_indices)):
					index_j = sorted_indices[j]

					if bbox_iou(boxes[index_i], boxes[index_j]) >= nms_threshold:
						boxes[index_j].classes[c] = 0

	# remove the boxes which are less likely than a obj_threshold
	boxes = [box for box in boxes if box.get_score() > obj_threshold]

	result = []
	for i in range(len(boxes)):
		if(boxes[i].classes[0] == 0):
			continue
		predicted_class = "face"
		score = boxes[i].score
		result.append([predicted_class, (boxes[i].xmax+boxes[i].xmin)*img_width/2, (boxes[i].ymax+boxes[i].ymin)
                 * img_height/2, (boxes[i].xmax-boxes[i].xmin)*img_width, (boxes[i].ymax-boxes[i].ymin)*img_height, score])

	return result

Overwriting boundingbox.py


In [18]:
%%writefile FaceDetector.py


from keras.models import load_model
import numpy as np
import sys
import cv2
import os
from keras import backend as K
import time
from boundingbox import interpret_output_yolov2, crop
os.environ['KERAS_BACKEND'] = 'tensorflow'

class FaceDetector:
    
    def __init__(self, model_path):
      self.detector = load_model(model_path)
    
    def detect_faces(self,frame):
        
        t = time.time()

        frame_resized = cv2.resize(frame, (416,416))/255.0
        frame_resized = np.expand_dims(frame_resized, axis=0)
        
        predict = self.detector.predict(frame_resized)[0]
        
        results = interpret_output_yolov2(predict, np.shape(frame)[1], np.shape(frame)[0])
        
        list_faces = []
        for i in range(len(results)):
          
            if results[i][5] >= 0.5 and results[i][0] == 'face':
                #display detected face
                x = int(results[i][1])
                y = int(results[i][2])
                w = int(results[i][3])//2
                h = int(results[i][4])//2
                
                xmin, xmax, ymin, ymax = crop(
                    x, y, w, h, 1.4, np.shape(frame)[1], np.shape(frame)[0])
                
                print(str(i)+'--  '+str(xmin)+'-'+str(xmax)+'-'+str(ymin)+'-'+str(ymax))
                
                list_faces.append(frame[ymin:ymax, xmin:xmax])

        print('detect face time:{0:.4f}'.format(time.time()-t))

        return list_faces
        
    def check_faces(self,list_faces):
      
        checked_faces = [face for face in list_faces if face.shape[0] >=IMAGE_SIZE_THESHOLD and face.shape[1] >=IMAGE_SIZE_THESHOLD]
        
        return checked_faces
        

Overwriting FaceDetector.py


In [19]:
%%writefile demo.py


from FaceDetector import FaceDetector
import cv2
import time
import numpy as np
model = FaceDetector('pretrain/yolov2_tiny-face.h5')

cap = cv2.VideoCapture('/content/drive/My Drive/module/video.mp4')
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
#Detection
while True:
    #Face Detection
    ret, frame = cap.read() #BGR

    #frame = cv2.imread("images/dress3.jpg")
    start_time = time.time()
    img=frame
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = model.detect_faces(img)
    print('detect face: '+str(time.time()-start_time)+' (s)')

    #cv2.imshow('image', frame)
    
#     if len(results)>0:
#         for i, result in enumerate(results):
#             cv2.imshow('face-'+str(i), cv2.cvtColor(result, cv2.COLOR_BGR2RGB))

    k = cv2.waitKey(1)
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

Overwriting demo.py


In [20]:
!python demo.py

Using TensorFlow backend.
W0723 08:14:43.383486 139644353337216 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0723 08:14:43.400288 139644353337216 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0723 08:14:43.422118 139644353337216 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0723 08:14:43.422299 139644353337216 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0723